In [1]:
using Pkg
Pkg.activate("..")
using Debugger


# Distributed setup
using ProgressiveHedging
const PH = ProgressiveHedging
using Ipopt
using JuMP, Xpress, PowerSimulations, PowerSystems, PowerSystemCaseBuilder, TimeSeries
const PSI = PowerSimulations
const PSY = PowerSystems
const PSB = PowerSystemCaseBuilder

# Local setup
using JuMP, Xpress, PowerSimulations, PowerSystems, PowerSystemCaseBuilder, InfrastructureSystems, Dates, TimeSeries
const PSI = PowerSimulations
const PSY = PowerSystems
const PSB = PowerSystemCaseBuilder

solver = optimizer_with_attributes(Xpress.Optimizer, "MIPRELSTOP" => 1e-4, "OUTPUTLOG" => 1, "MAXTIME" => 500,)

sys_name = "/lustre/eaglefs/projects/pvb/cju/gen_systems/data/rts_with_battery_060822_sys.json"
system = PSY.System(sys_name, time_series_directory = "/tmp/scratch")
PSY.transform_single_time_series!(system, 672, Hour(168))
# PSY.transform_single_time_series!(system, #Horizon, #Interval)

template_ed = ProblemTemplate(PowerSimulations.DCPPowerModel)
# Device models
set_device_model!(template_ed, PowerSystemCaseBuilder.Line, StaticBranch)
set_device_model!(template_ed, PowerSystemCaseBuilder.Transformer2W, StaticBranch)
set_device_model!(template_ed, PowerSystemCaseBuilder.TapTransformer, StaticBranch)

# Other systems
set_device_model!(template_ed, PowerSystems.ThermalStandard, ThermalBasicUnitCommitment)
set_device_model!(template_ed, PowerSystems.RenewableDispatch, RenewableFullDispatch)
set_device_model!(template_ed, PowerSystems.PowerLoad, StaticPowerLoad)
set_device_model!(template_ed, PowerSystems.HydroDispatch, FixedOutput)
set_device_model!(template_ed, PowerSystems.GenericBattery, BookKeeping)
set_device_model!(template_ed, PowerSystems.RenewableFix, FixedOutput)

# Ancillary services
# set_service_model!(template_ed, VariableReserve{ReserveUp}, RangeReserve)
# set_service_model!(template_ed, VariableReserve{ReserveDown}, RangeReserve)

problem = DecisionModel(
    template_ed, 
    system,
    horizon = 72,
    name = "SubProblem",
    optimizer = solver, 
    warm_start = false,
    export_pwl_vars = true,
    initialize_model = false,
    initial_time = DateTime("2020-01-01T00:00:00"),
)

directory ="./simulation_folder/"
build!(problem, output_dir = directory)
solve!(problem)

res = ProblemResults(problem)

  Activating project at `/lustre/eaglefs/projects/pvb/cju/ProgressiveHedging.jl`
┌ Info: Xpress: Found license file /nopt/nrel/apps/xpressmp/8.13.0/bin/xpauth.xpr
└ @ Xpress /home/cju/.julia/packages/Xpress/xOQbX/src/license.jl:44
┌ Info: Xpress: Development license detected.
└ @ Xpress /home/cju/.julia/packages/Xpress/xOQbX/src/license.jl:89
┌ Info: Loaded time series from storage file existing=rts_with_battery_060822_sys_time_series_storage.h5 new=/tmp/scratch/jl_6rf0Zx compression=CompressionSettings(false, CompressionTypes.DEFLATE = 1, 3, true)
└ @ InfrastructureSystems /home/cju/.julia/packages/InfrastructureSystems/Oc56m/src/hdf5_time_series_storage.jl:100


ProductionCostExpression__RenewableDispatch
ProductionCostExpression__ThermalStandard
ActivePowerTimeSeriesParameter__PowerLoad
ActivePowerTimeSeriesParameter__RenewableFix
ActivePowerTimeSeriesParameter__RenewableDispatch
ActivePowerTimeSeriesParameter__HydroDispatch
EnergyVariable__GenericBattery
FlowActivePowerVariable__Line
ActivePowerVariable__RenewableDispatch
VoltageAngle__Bus
ReservationVariable__GenericBattery


In [3]:
methodswith(typeof(res))

[1] export_optimizer_stats(res::Union{ProblemResults, PowerSimulations.SimulationProblemResults}, directory::AbstractString; format) in PowerSimulations at /home/cju/.julia/packages/PowerSimulations/HKwUO/src/simulation/simulation_problem_results.jl:688
[2] export_realized_results(res::ProblemResults) in PowerSimulations at /home/cju/.julia/packages/PowerSimulations/HKwUO/src/operation/problem_results.jl:790
[3] export_realized_results(res::Union{ProblemResults, PowerSimulations.SimulationProblemResults}, save_path::AbstractString) in PowerSimulations at /home/cju/.julia/packages/PowerSimulations/HKwUO/src/simulation/simulation_problem_results.jl:647
[4] export_results(results::ProblemResults; kwargs...) in PowerSimulations at /home/cju/.julia/packages/PowerSimulations/HKwUO/src/operation/problem_results.jl:136
[5] export_results(results::ProblemResults, exports::ProblemResultsExport; file_type) in PowerSimulations at /home/cju/.julia/packages/PowerSimulations/HKwUO/src/operation/problem_results.jl:147
[6] get_aux_variable_values(res::ProblemResults) in PowerSimulations at /home/cju/.julia/packages/PowerSimulations/HKwUO/src/operation/problem_results.jl:36
[7] get_dual_values(res::ProblemResults) in PowerSimulations at /home/cju/.julia/packages/PowerSimulations/HKwUO/src/operation/problem_results.jl:33
[8] get_expression_values(res::ProblemResults) in PowerSimulations at /home/cju/.julia/packages/PowerSimulations/HKwUO/src/operation/problem_results.jl:34
[9] get_model_base_power(res::ProblemResults) in PowerSimulations at /home/cju/.julia/packages/PowerSimulations/HKwUO/src/operation/problem_results.jl:32
[10] get_objective_value(res::ProblemResults) in PowerSimulations at /home/cju/.julia/packages/PowerSimulations/HKwUO/src/operation/problem_results.jl:44
[11] get_objective_value(res::ProblemResults, execution) in PowerSimulations at /home/cju/.julia/packages/PowerSimulations/HKwUO/src/operation/problem_results.jl:44
[12] get_optimizer_stats(res::ProblemResults) in PowerSimulations at /home/cju/.julia/packages/PowerSimulations/HKwUO/src/operation/problem_results.jl:38
[13] get_parameter_values(res::ProblemResults) in PowerSimulations at /home/cju/.julia/packages/PowerSimulations/HKwUO/src/operation/problem_results.jl:39
[14] get_resolution(res::ProblemResults) in PowerSimulations at /home/cju/.julia/packages/PowerSimulations/HKwUO/src/operation/problem_results.jl:40
[15] get_system(res::ProblemResults) in PowerSimulations at /home/cju/.julia/packages/PowerSimulations/HKwUO/src/operation/problem_results.jl:41
[16] get_timestamps(res::ProblemResults) in PowerSimulations at /home/cju/.julia/packages/PowerSimulations/HKwUO/src/operation/problem_results.jl:31
[17] get_variable_values(res::ProblemResults) in PowerSimulations at /home/cju/.julia/packages/PowerSimulations/HKwUO/src/operation/problem_results.jl:35
[18] list_aux_variable_keys(res::ProblemResults) in PowerSimulations at /home/cju/.julia/packages/PowerSimulations/HKwUO/src/operation/problem_results.jl:18
[19] list_aux_variable_names(res::ProblemResults) in PowerSimulations at /home/cju/.julia/packages/PowerSimulations/HKwUO/src/operation/problem_results.jl:19
[20] list_dual_keys(res::ProblemResults) in PowerSimulations at /home/cju/.julia/packages/PowerSimulations/HKwUO/src/operation/problem_results.jl:26
[21] list_dual_names(res::ProblemResults) in PowerSimulations at /home/cju/.julia/packages/PowerSimulations/HKwUO/src/operation/problem_results.jl:27
[22] list_expression_keys(res::ProblemResults) in PowerSimulations at /home/cju/.julia/packages/PowerSimulations/HKwUO/src/operation/problem_results.jl:28
[23] list_expression_names(res::ProblemResults) in PowerSimulations at /home/cju/.julia/packages/PowerSimulations/HKwUO/src/operation/problem_results.jl:29
[24] list_parameter_keys(res::ProblemResults) in PowerSimulations at /home/cju/.julia/packages/PowerSimulations/HKwUO/src/operation/problem_results.jl:23
[25] list_parameter_names(res::ProblemResults) in PowerSimulati

In [4]:
get_objective_value(res)

2.275470590130735e6